#### Import

In [157]:
#!pip install pingouin

In [158]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import calendar
#import pingouin as pg
from datetime import datetime
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [159]:
from google.colab import drive
drive.mount('/content/gdrive')

# directory
%cd '/content/gdrive/My Drive/Oxford/ML_for_Social_Good'

# import
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

train["Set"] = "train"
test["Set"] = "test"

df = pd.concat([train, test])
print(df.shape)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Oxford/ML_for_Social_Good
(5160, 45)


#### Basic adjustments

In [160]:
datetime_cols = ["CropTillageDate","RcNursEstDate","SeedingSowingTransplanting","Harv_date","Threshing_date"]

for col in datetime_cols:
  df[col] = pd.to_datetime(df[col])#.dt.date

In [161]:
# One row has Jamui as district but Gurua as Block, which is a Gaya block -- correcting its District
df.loc[(df["District"]=="Jamui") & (df["Block"]=="Gurua")].index
df.loc[2177,"District"] = "Gaya"

#### Outliers

In [162]:
# OUTLIERS -- first pass

# SeedlingsPerPit has two extreme outliers (800 seedlings & 442 seedlings) -- replacing with the next max value (22)
#df["SeedlingsPerPit"] = df["SeedlingsPerPit"].replace(800,22).replace(442,22)
df.loc[df["SeedlingsPerPit"]>22, "SeedlingsPerPit"] = 22

# TransplantingIrrigationHours -- capping at 450
df.loc[df["TransplantingIrrigationHours"]>450, "TransplantingIrrigationHours"] = 450

# TransIrriCost have several extreme outliers (e.g. 6000 rupees for an average sized land) -- capping at 3000
df.loc[df["TransIrriCost"]>3000, "TransIrriCost"] = 3000

# Ganaura -- making a capped version at 50 (already way above the upper fence), and leaving the raw variable to compare
df.loc[df["Ganaura"]>50, "Ganaura_capped"] = 50

# 1appDaysUrea -- replacing extreme outlier with the next max value
df["1appDaysUrea"] = df["1appDaysUrea"].replace(332,75)

# Harv_hand_rent -- capping at 20000 (there are 2 values above that) (upper fence is at 1500, so 20000 is conservative)
df.loc[df["Harv_hand_rent"]>20000, "Harv_hand_rent"] = 20000

#### Per_Acre columns

In [163]:
# PER-ACRE COLUMNS

list_cols = ["TransIrriCost","Ganaura","CropOrgFYM","BasalDAP","BasalUrea","1tdUrea","2tdUrea","Harv_hand_rent","Yield"]

for col in list_cols:
  label = str(col) + "_per_Acre"
  df[label] = df[col] / df["Acre"]

In [164]:
# Re-ordering columns and dropping the non-standardized ones
df = df[['ID', 'Set', 'District', 'Block', 'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth','CropEstMethod', 'RcNursEstDate',
         'SeedingSowingTransplanting','SeedlingsPerPit', 'NursDetFactor', 'TransDetFactor','TransplantingIrrigationHours', 'TransplantingIrrigationSource',
         'TransplantingIrrigationPowerSource', "TransIrriCost", 'TransIrriCost_per_Acre', 'StandingWater','OrgFertilizers', 'Ganaura', 'Ganaura_per_Acre', 'CropOrgFYM',
         'CropOrgFYM_per_Acre', 'PCropSolidOrgFertAppMethod', 'NoFertilizerAppln', 'CropbasalFerts', "BasalDAP", 'BasalDAP_per_Acre', "BasalUrea", 'BasalUrea_per_Acre',
         'MineralFertAppMethod', 'FirstTopDressFert', "1tdUrea",'1tdUrea_per_Acre', '1appDaysUrea', "2tdUrea", '2tdUrea_per_Acre', '2appDaysUrea', 'MineralFertAppMethod.1',
         'Harv_method', 'Harv_date', "Harv_hand_rent", 'Harv_hand_rent_per_Acre', 'Threshing_date', 'Threshing_method','Residue_length', 'Residue_perc', 'Stubble_use',
         'Acre', 'Yield','Yield_per_Acre'
         ]]

#### Parsing messy categorical variables

In [165]:
# PARSING MESSY CATEGORICAL VARIABLES

# 1. LandPreparationMethod
#methods = ["TractorPlough","FourWheelTracRotavator","WetTillagePuddling","BullockPlough","Other"]

df["LandPrepMethod_TractorPlough"] = df["LandPreparationMethod"].str.contains("TractorPlough")
df["LandPrepMethod_FourWheelTracRotavator"] = df["LandPreparationMethod"].str.contains("FourWheelTracRotavator")
df["LandPrepMethod_WetTillagePuddling"] = df["LandPreparationMethod"].str.contains("WetTillagePuddling")
df["LandPrepMethod_BullockPlough"] = df["LandPreparationMethod"].str.contains("BullockPlough")
df["LandPrepMethod_Other"] = df["LandPreparationMethod"].str.contains("Other")


# 2. NursDetFactor
#reasons = ["CalendarDate","PreMonsoonShowers","IrrigWaterAvailability","LabourAvailability","SeedAvailability"]

df["NursDetFactor_CalendarDate"] = df["NursDetFactor"].str.contains("CalendarDate")
df["NursDetFactor_PreMonsoonShowers"] = df["NursDetFactor"].str.contains("PreMonsoonShowers")
df["NursDetFactor_IrrigWaterAvailability"] = df["NursDetFactor"].str.contains("IrrigWaterAvailability")
df["NursDetFactor_LabourAvailability"] = df["NursDetFactor"].str.contains("LabourAvailability" or "LaborAvailability")
df["NursDetFactor_SeedAvailability"] = df["NursDetFactor"].str.contains("SeedAvailability")


# 2. TransDetFactor
#reasons = ["LaborAvailability","CalendarDate","RainArrival","IrrigWaterAvailability","SeedlingAge"] # I think that's all of them

df["TransDetFactor_LabourAvailability"] = df["TransDetFactor"].str.contains("LabourAvailability" or "LaborAvailability")
df["TransDetFactor_CalendarDate"] = df["TransDetFactor"].str.contains("CalendarDate")
df["TransDetFactor_RainArrival"] = df["TransDetFactor"].str.contains("RainArrival")
df["TransDetFactor_IrrigWaterAvailability"] = df["TransDetFactor"].str.contains("IrrigWaterAvailability")
df["TransDetFactor_SeedlingAge"] = df["TransDetFactor"].str.contains("SeedlingAge")


# 3. CropbasalFerts
df["CropbasalFerts"] = df["CropbasalFerts"].fillna("None")
fertilizer_types = ["Urea","DAP","Other","NPK","MoP","NPKS","SSP","None"]

for fertilizer in fertilizer_types:
  label = "CropbasalFerts_" + fertilizer
  df[label] = df["CropbasalFerts"].str.contains(fertilizer)


# 4. FirstTopDressFert
df["FirstTopDressFert"] = df["FirstTopDressFert"].fillna("None")
fertilizer_types2 = ["Urea","DAP","NPK","NPKS","SSP","Other"]

for fertilizer in fertilizer_types2:
  label = "FirstTopDressFert_" + fertilizer
  df[label] = df["FirstTopDressFert"].str.contains(fertilizer)


# 5. OrgFertilizers
df["OrgFertilizers"] = df["OrgFertilizers"].fillna("None")
orgfertilizers = ["Ganaura","FYM","VermiCompost","Pranamrit","Ghanajeevamrit","Jeevamrit","PoultryManure"]
for fertilizer in orgfertilizers:
  label = "OrgFertilizers_" + fertilizer
  df[label] = df["OrgFertilizers"].str.contains(fertilizer)


# 6. Replacing all NaNs with False
cols = ['LandPrepMethod_TractorPlough','LandPrepMethod_FourWheelTracRotavator','LandPrepMethod_WetTillagePuddling', 'LandPrepMethod_BullockPlough','LandPrepMethod_Other',
        'NursDetFactor_CalendarDate','NursDetFactor_PreMonsoonShowers','NursDetFactor_IrrigWaterAvailability','NursDetFactor_LabourAvailability', 'NursDetFactor_SeedAvailability',
        'TransDetFactor_LabourAvailability', 'TransDetFactor_CalendarDate','TransDetFactor_RainArrival', 'TransDetFactor_IrrigWaterAvailability','TransDetFactor_SeedlingAge',
        'CropbasalFerts_Urea','CropbasalFerts_DAP', 'CropbasalFerts_Other', 'CropbasalFerts_NPK','CropbasalFerts_MoP', 'CropbasalFerts_NPKS', 'CropbasalFerts_SSP',
        'CropbasalFerts_None', 'FirstTopDressFert_Urea','FirstTopDressFert_DAP', 'FirstTopDressFert_NPK','FirstTopDressFert_NPKS', 'FirstTopDressFert_SSP','FirstTopDressFert_Other',
        'OrgFertilizers_Ganaura','OrgFertilizers_FYM', 'OrgFertilizers_VermiCompost','OrgFertilizers_Pranamrit', 'OrgFertilizers_Ghanajeevamrit','OrgFertilizers_Jeevamrit',
        'OrgFertilizers_PoultryManure']

for col in cols:
  df[col] = df[col].fillna(False)

#### Missing values

In [166]:
# For 1appDaysUrea, 2appDaysUrea, in most cases NaN means there was so 2nd or 3rd dose, so NaN is appropriate.
# For a couple rows, however, there was a 2nd or 3rd dose (as indicated by 1tdUrea and 2tdUrea) but the number of days value is missing, in which case they probably need imputation

# For rows where XappDaysUrea is NaN but XtdUrea is not NaN, impute with block median
subset = df.loc[(df["Block"]=="Rajgir")]
df.loc[(df["1appDaysUrea"].isnull()==True) & (df["1tdUrea"].isnull()==False), "1appDaysUrea"] = subset["1tdUrea"].median()

subset = df.loc[(df["Block"]=="Gurua")]
df.loc[(df["2appDaysUrea"].isnull()==True) & (df["2tdUrea"].isnull()==False), "2appDaysUrea"] = subset["2tdUrea"].median()

# Imputing with full sample medians
#df.loc[(df["1appDaysUrea"].isnull()==True) & (df["1tdUrea"]!=0), "1appDaysUrea"] = df["1appDaysUrea"].median()
#df.loc[(df["2appDaysUrea"].isnull()==True) & (df["2tdUrea"]!=0), "2appDaysUrea"] = df["2appDaysUrea"].median()


In [167]:
# Replacing NaN with 0 for columns where it makes sense
fillna0 = ["2tdUrea","1tdUrea","Harv_hand_rent","Ganaura","CropOrgFYM","BasalDAP","BasalUrea"]
for col in fillna0:
  df[col] = df[col].fillna(0)

# Creating a new variable counting the number of missing values for each row (excluding outcome variables)
df["Nb_of_NaN"] = df.drop(columns=["Yield","Yield_per_Acre"]).isnull().sum(axis=1)
# in case that's useful to define a threshold and drop rows that are too incomplete
print(df.loc[df["Nb_of_NaN"]>10].shape[0], "rows have over 10 missing values")
# updated thought: a bunch of NaNs are actually meaningful, so I don't think defining a threshold is useful

225 rows have over 10 missing values


In [168]:
# For TransplantingIrrigationHours, TransplantingIrrigationSource, and TransplantingIrrigationPowerSource; no significant statistical difference between NaNs and non-NaNs on yields.

print("TransplantingIrrigationSource - Number of NaNs before imputation: ", df["TransplantingIrrigationSource"].isnull().sum())
print("TransplantingIrrigationPowerSource - Number of NaNs before imputation: ", df["TransplantingIrrigationPowerSource"].isnull().sum())
print("TransplantingIrrigationHours - Number of NaNs before imputation: ", df["TransplantingIrrigationHours"].isnull().sum())

# For TransplantingIrrigationSource and TransplantingIrrigationPowerSource, imputing missing values with most common category (mode)
df.loc[df["TransplantingIrrigationSource"].isnull()==True, "TransplantingIrrigationSource"] = df["TransplantingIrrigationSource"].mode()
df.loc[df["TransplantingIrrigationPowerSource"].isnull()==True, "TransplantingIrrigationPowerSource"] = df["TransplantingIrrigationPowerSource"].mode()

# For TransplantingIrrigationHours, imputing missing values with the median
df.loc[df["TransplantingIrrigationHours"].isnull()==True, "TransplantingIrrigationHours"] = df["TransplantingIrrigationHours"].median()

TransplantingIrrigationSource - Number of NaNs before imputation:  161
TransplantingIrrigationPowerSource - Number of NaNs before imputation:  674
TransplantingIrrigationHours - Number of NaNs before imputation:  270


Remaining variables with NaNs:
- RcNursEstDate: statistically significant difference between rows with missing values vs. not in this variable, so leaving NaNs alone for now
- SeedlingsPerPit: same thing
- NursDetFactor: same thing
- TransIrriCost: same thing
- StandingWater: same thing

#### Exporting cleaned df

In [169]:
# Dropping messy cols
df = df.drop(columns=["LandPreparationMethod","NursDetFactor","TransDetFactor","OrgFertilizers","CropbasalFerts","FirstTopDressFert"])

In [170]:
# EXPORTING

# V1: full cleaned df
df.to_csv('cleaned_fulldf.csv',index=False)

# V2: per Acre df (dropping the raw variables)
#df.copy().drop(columns=["TransIrriCost","Ganaura","CropOrgFYM","BasalDAP","BasalUrea","1tdUrea","2tdUrea","Harv_hand_rent","Yield"]).to_csv('peracre_df.csv',index=False)

# V2: per Acre df (dropping the raw variables)
#df.copy().drop(columns=["TransIrriCost_per_Acre","Ganaura_per_Acre","CropOrgFYM_per_Acre","BasalDAP_per_Acre","BasalUrea_per_Acre","1tdUrea_per_Acre","2tdUrea_per_Acre",
#                        "Harv_hand_rent_per_Acre","Yield_per_Acre"]).to_csv('rawyield_df.csv',index=False)

In [171]:
df.shape

(5160, 83)